# Materia prima, costos y ventas

Crearemos un dataframe que sea de utilidad para hacer un analisis enfocado en analizar el detalle de todos los productos vendidos. El dataframe madre contiene todos los registros y columnas obtenidas por cada articulo incluido en una orden de compra.

In [285]:
import pandas as pd
import numpy as np

In [286]:
# Importamos el dataframe generado desde el kernel ubicado en '/..pymongo/raw'. El kernel inserta el csv directamente aqui.

df = pd.read_csv('operaciones.csv').reset_index(drop=True)

# Para añadir la columna de precios de fabrica, añadimos tambien la tabla de precios vigentes que asigna un precio a partir del calibre, la estructura
# y el tipo de cara Exterior

prc = pd.read_csv('../recursos/listaPrecios2020.csv', sep=';' ).reset_index(drop=True)

Usamos el metodo **_.merge()_** para unir el dataframe original con la tabla de precios de lista **_'../recursos/listaPrecios2020.csv'_**

In [287]:
precios = df.merge(prc, how='left', left_on=['Especificaciones.calibreECT', 'Especificaciones.caraExterior', 'Especificaciones.estructura'], right_on=['precios.calibreECT', 'precios.caraExterior', 'precios.estructura'])

#Verificamos la relación en algunos valores al azar

precios.loc[154]

Unnamed: 0                                                               154
_id                                                 5fcb95d34124c8597eb6c8c5
idUsuario                                           5fba998a345a4bb6ed3446c8
tipo                                                         ordenProduccion
estado                                                               abierta
foliointerno                                                           24152
fechaEmision                                                      2020-09-11
fechaCierre                                                       2020-10-30
priceKey                                                      29regularkraft
Productos.idProducto                                5fc9d875450966d58fee1ef4
Productos.idLocal                                                       CJ61
Productos.unidades                                                  millares
Productos.precioUnitario                                                5512

In [288]:
# Creamos una lista con las columnas que vamos a necesitar

material = ['foliointerno',
            '_id',
'Usuario.razonSocial', 
'Productos.idLocal',
'Productos.unidades',
'Productos.cantidad',
'Productos.importe',
'Especificaciones.estructura',
'Especificaciones.tipo',
'Especificaciones.calibreECT',
'Especificaciones.caraExterior',
'Especificaciones.largo',
'Especificaciones.ancho',
'Especificaciones.alto',
'Especificaciones.tipoUnion',
'Especificaciones.suaje',
'Productos.precioUnitario',
            'precios.precioGeneralm2',
            'fechaEmision'
           ]

# Introducimos el diccionario como filtro para ordenar

material = precios[material]
material

,foliointerno,_id,Usuario.razonSocial,Productos.idLocal,Productos.unidades,Productos.cantidad,Productos.importe,Especificaciones.estructura,Especificaciones.tipo,Especificaciones.calibreECT,Especificaciones.caraExterior,Especificaciones.largo,Especificaciones.ancho,Especificaciones.alto,Especificaciones.tipoUnion,Especificaciones.suaje,Productos.precioUnitario,precios.precioGeneralm2,fechaEmision
0,24156,5fc3da6c62f23e3d63860a22,LOQUAY. S.A.,CJ246,millares,2.0,21200.0,caja regular,regular,32,kraft,32.3,24.5,30.3,CIG,0,10600.0,14.12,2020-09-11
1,24157,5fc3db5f62f23e3d63860a24,LOQUAY. S.A.,CJ245,millares,2.5,8840.0,caja regular,regular,29,kraft,19.6,15.0,17.5,CIP,0,3536.0,13.25,2020-09-11
2,24374,5fc3de2962f23e3d63860a26,LOQUAY. S.A.,CJ247,millares,2.0,27664.0,caja en dos piezas,dosPiezas,32,kraft,44.3,32.0,14.5,CIP,0,13832.0,18.30,2020-11-06
3,24375,5fc3dfbc62f23e3d63860a27,LOQUAY. S.A.,CJ65,millares,3.0,8268.0,caja regular,regular,29,kraft,20.0,10.4,18.5,CIP,0,2756.0,13.25,2020-11-06
4,24377,5fc3e09a62f23e3d63860a28,LOQUAY. S.A.,CJ67,millares,2.0,8382.0,caja regular,dosPiezas,29,kraft,26.3,17.8,13.2,CIP,0,4191.0,13.25,2020-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,23197,5fcbf2174124c8597eb6c90c,LOQUAY. S.A.,CJ90,millares,2.0,5512.0,caja regular,regular,29,kraft,20.0,10.4,18.5,CIP,0,2756.0,13.25,2020-01-28
231,23198,5fcbf24f4124c8597eb6c90d,LOQUAY. S.A.,CJ67,millares,5.0,20955.0,caja regular,dosPiezas,29,kraft,26.3,17.8,13.2,CIP,0,4191.0,13.25,2020-01-28
232,23200,5fcbf28a4124c8597eb6c90e,LOQUAY. S.A.,CJ249,millares,2.0,20176.0,caja regular,regular,36,kraft,32.8,25.0,25.3,CIG,0,10088.0,14.64,2020-01-28
233,23201,5fcbf2e24124c8597eb6c90f,LOQUAY. S.A.,CJ85,millares,5.0,18720.0,caja regular,regular,29,kraft,19.6,15.0,20.0,CIP,0,3744.0,13.25,2020-01-28


El corrugado se construye a partir de m2 de lamina producida de un mismo calibre por lo que añadiremos las columnas necesarias para calcular el area de la lamina en m2, segun la siguiente formula:

               L(lamina) = ((2*L) * (2*W)) + desarrollo
               W(lamina) = (L*W) + desarrollo
               Área en m2 = L(lamina) * W(lamina)
                    
La variable de desarrollo se asigna dependiendo del tipo de corrugado de acuerdo a la siguiente tabla:

                    {'regular': 5.5,
                    'dosPiezas': 1.5,
                    'N/A': 0}
              

In [289]:
# Creamos el diccionario para sustituir la llave del tipo con el desarrollo de tolerancia.

desarrollo = {'regular': [5.5, 1.5],
                    'dosPiezas': [7, 2.7],
                    np.nan: [0, 0]}

# Sustituyendo y aplicando el diccionario en la columna

material["Especificaciones.tipo"] = material["Especificaciones.tipo"].map(desarrollo)

<ipython-input-289-fdd7339171be>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.tipo"] = material["Especificaciones.tipo"].map(desarrollo)


In [290]:
# Crear funciones para calcular la longitud y ancho totales de la lamina

def largoLaminaM2(largo, ancho, desarrollo):
    longitudLamina = ((2*largo) + (2*ancho)) + desarrollo
    return longitudLamina / 100

def anchoLaminaM2(ancho, alto, desarrollo):
    anchoLamina = (ancho + alto + desarrollo)
    return anchoLamina / 100

### Creamos las columnas nuevas aplicando las funciones

In [291]:
# Largo de lamina

material["Especificaciones.longitudLamina"] = largoLaminaM2(material["Especificaciones.largo"], material["Especificaciones.ancho"], material["Especificaciones.tipo"].str[0])

# Ancho de lamina

material["Especificaciones.anchoLamina"] = anchoLaminaM2(material["Especificaciones.ancho"], material["Especificaciones.alto"], material["Especificaciones.tipo"].str[1])

# Calculamos el área total en metros cuadrados por cada producto vendido.

material["areaM2"] = material["Especificaciones.anchoLamina"] * material["Especificaciones.longitudLamina"]
material["precioListaMillares"] = (material["areaM2"] * material["precios.precioGeneralm2"])*1000

<ipython-input-291-ec2e7bd9e75b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.longitudLamina"] = largoLaminaM2(material["Especificaciones.largo"], material["Especificaciones.ancho"], material["Especificaciones.tipo"].str[0])
<ipython-input-291-ec2e7bd9e75b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material["Especificaciones.anchoLamina"] = anchoLaminaM2(material["Especificaciones.ancho"], material["Especificaciones.alto"], material["Especificaciones.tipo"].str[1])
<ipy

In [293]:
material.describe()

,Productos.cantidad,Productos.importe,Especificaciones.calibreECT,Especificaciones.largo,Especificaciones.ancho,Especificaciones.alto,Especificaciones.suaje,Productos.precioUnitario,precios.precioGeneralm2,Especificaciones.longitudLamina,Especificaciones.anchoLamina,areaM2,precioListaMillares
count,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000
mean,3.617979,42776.140426,30.093617,67.884681,37.010213,18.679149,0.225532,18838.851064,10.489872,2.137877,0.568268,1.407369,14601.042294
std,1.729977,40372.938732,3.084245,56.177055,18.063378,32.951930,0.418824,26048.850446,3.699597,1.446636,0.388670,1.361098,16020.506817
min,0.100000,5512.000000,23.000000,19.000000,10.400000,0.000000,0.000000,2720.000000,5.800000,0.663000,0.290000,0.201552,1951.120000
25%,2.000000,13600.000000,27.500000,29.000000,24.000000,0.000000,0.000000,2756.000000,7.500000,1.126000,0.304000,0.336400,2670.564000
50%,4.000000,21000.000000,32.000000,32.300000,29.000000,13.300000,0.000000,10088.000000,13.250000,1.211000,0.495000,0.637185,9183.642720
75%,5.000000,78800.000000,32.000000,104.000000,59.100000,20.000000,0.000000,21685.000000,14.120000,2.786000,0.627000,2.867271,21504.532500
max,7.000000,189000.000000,38.000000,164.700000,95.700000,200.000000,1.000000,123000.000000,18.300000,4.573000,2.630000,7.471830,105502.239600


Exportamos este dataframe unificado para utilizarlo como fuente base de información.

In [271]:
material.to_csv('material.csv')